In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import numpy as np
import time
import pandas as pd
import re, string, sys
from textblob import TextBlob

import utils.global_vars as gv
from utils.stop_words import StopWords
from utils.config import Config as config
from utils import stop_words

from training_data.data_prepare import DataPrepare
from training_data.generate_training_data import GenerateTrainingData
from training_data.labeling_functions import LabelData
from termcolor import colored

import spacy
from nlp_pipeline.nouns_pipeline import PatentNLPPreprocessor
from nlp_pipeline import svo_pipeline
import pickle
import itertools

In [2]:
data=DataPrepare()

data.set_der_abst()
data.set_full_text()

DataPrepare object has the following properties:
 df_full_text, dict_full_text, df_der_text, dict_der_text


KeyboardInterrupt: 

In [ ]:
for 

In [4]:
obj_gen_training_data=GenerateTrainingData(data.df_full_text, data.dict_full_text, data.df_der_text, 
                                           data.dict_der_text, "noun_phrase")

In [5]:
df_kw, df_res= obj_gen_training_data.label_data()

0 --- 13.31009817123413 seconds ---
10 --- 167.51785898208618 seconds ---
20 --- 108.96171379089355 seconds ---
30 --- 82.69539880752563 seconds ---
40 --- 56.63676691055298 seconds ---
50 --- 37.92247009277344 seconds ---
60 --- 103.05678176879883 seconds ---
70 --- 46.2660071849823 seconds ---
80 --- 47.970197916030884 seconds ---
90 --- 29.45009708404541 seconds ---
No itersection elements found count: 0
count_key_not_in_dict: 0
count_pass_kw: 0
index: 99


In [6]:
df_kw.head()

,PPID,KEYWORD
0,CA2798250A1,viable circulate stem cell
1,CA2798250A1,circulate stem cell
2,CA2798250A1,carcinoma
3,CA2798250A1,viable form
4,CA2798250A1,colorectal carcinoma


In [8]:
list_kw=[]
for index, (reviewerID, hist) in enumerate (df_kw.groupby('PPID')):    
    list_kw+=hist['KEYWORD'].tolist()

In [7]:
list(list(df_kw.groupby('PPID'))[1][2]['KEYWORD'])

IndexError: tuple index out of range

In [ ]:
list(list(df_kw.groupby('PPID'))[1][2]['KEYWORD'])

In [9]:
dict_occ=dict((x,list_kw.count(x)) for x in set(list_kw))
dict_occ_sorted={k: v for k, v in sorted(dict_occ.items(), key=lambda item: item[1])}

In [ ]:
def get_labels(self, text):

    if self.labelling_function=="noun_phrase":
        return self.nlp_pipeline.run_NLP(text, lemma_normalize=True)

In [21]:
print(spacy.__version__) #2.3.0

spacy_NLP_model = spacy.load('en_core_web_sm', disable=['ner'])

nlp_pipeline = PatentNLPPreprocessor(spacy_NLP_model)



rows_kw = []
rows_tit_abst_clem = []
count_key_not_in_dict=0
count_no_itersec=0
count_pass_kw=0

start_time = time.time()

for i, (key, value) in enumerate(data.dict_der_text.items()):    

    if key in data.dict_full_text: 

        keyterms_derwent=set(nlp_pipeline.run_NLP(value, lemma_normalize=True))

        df_temp=data.df_full_text.loc[data.df_full_text['PPID'] == key]

        title=df_temp['TIEN'].iloc[0]
        abst=df_temp['ABEN'].iloc[0]
        claim=df_temp['CLMEN'].iloc[0]   
        detden=df_temp['DETDEN'].iloc[0] 

        count_cor_tien=nlp_pipeline.run_NLP(title, lemma_normalize=True)
        print("--- %s seconds title---" % (time.time() - start_time))
        start_time = time.time()
        
        count_cor_abst=nlp_pipeline.run_NLP(abst, lemma_normalize=True)
        print("--- %s seconds abst---" % (time.time() - start_time))
        start_time = time.time()
        
        count_cor_clem = nlp_pipeline.run_NLP(claim, lemma_normalize=True)
        print("--- %s seconds clem---" % (time.time() - start_time))
        start_time = time.time()
        
        count_cor_detden = nlp_pipeline.run_NLP(detden, lemma_normalize=True)
        print("--- %s seconds detden---" % (time.time() - start_time))
        start_time = time.time()

        set_labels_full_text=set(itertools.chain(count_cor_tien, count_cor_abst,count_cor_clem,count_cor_detden))

        set_intersec = keyterms_derwent.intersection(set_labels_full_text)  #common elments derwent and full text 

        if len(set_intersec)==0: 
            count_no_itersec+=1

        else:
            sub_list=set_intersec
            rows_tit_abst_clem.append([key, count_cor_tien, count_cor_abst, count_cor_clem, count_cor_detden])

            for kw in sub_list:
                rows_kw.append([key, kw])
                if kw not in set_labels_full_text :
                    print('Keyterm is not in intersection', can[0])
                    sys.exit()
    else:
        count_key_not_in_dict+=1


    if i%10==0:
        print(i,"--- %s seconds ---" % (time.time() - start_time))
        start_time = time.time()


print('No itersection elements found count:',count_no_itersec)
print('count_key_not_in_dict:',count_key_not_in_dict)
print('count_pass_kw:',count_pass_kw)
print('index:',i)

#print("Number of intersec elements is less than %s :",n_of_keywords_to_extract,count_less_kws)

df_kw = pd.DataFrame(rows_kw, columns=["PPID", "KEYWORD"])  
df_res = pd.DataFrame(rows_tit_abst_clem, columns=["PPID","TIEN", "ABEN","CLMEN","DETDEN"]) 



'''
terms = nlp_pipeline.run_NLP(text, lemma_normalize=True)

# Extract noun hrases as are in the original text
nouns = nlp_pipeline.get_noun_phrases(text)
doc = spacy_NLP_model(text1)

# Extract Subject-Verb-Object
svo_out= svo_pipeline.subject_verb_object_triples(doc)

print(list(svo_out))
print(list(terms))
print(nouns)

'''

2.3.0
--- 0.6114847660064697 seconds title---
--- 0.0876779556274414 seconds abst---
--- 1.8028042316436768 seconds clem---
--- 10.913206100463867 seconds detden---
0 --- 8.678436279296875e-05 seconds ---
--- 0.26973795890808105 seconds title---
--- 0.05341815948486328 seconds abst---
--- 0.8746981620788574 seconds clem---
--- 12.244946956634521 seconds detden---
--- 0.09316587448120117 seconds title---
--- 0.03930807113647461 seconds abst---
--- 0.45094823837280273 seconds clem---
--- 12.33269190788269 seconds detden---
--- 0.2551240921020508 seconds title---
--- 0.07553267478942871 seconds abst---
--- 2.1610870361328125 seconds clem---
--- 7.839497804641724 seconds detden---
--- 0.2791919708251953 seconds title---
--- 0.042842864990234375 seconds abst---
--- 0.6841797828674316 seconds clem---
--- 5.606541156768799 seconds detden---
--- 0.22480487823486328 seconds title---
--- 0.04337620735168457 seconds abst---
--- 0.2850770950317383 seconds clem---
--- 4.970601797103882 seconds detd

KeyboardInterrupt: 

In [13]:
with open('test.txt', 'wb') as handle:
    pickle.dump(dict_occ_sorted, handle)

In [15]:
with open('test.txt', 'rb') as handle:
    b = pickle.loads(handle.read())

{'pcr process': 1, 'place': 1, 'chinese toon cornus': 1, 'radix scutellariae': 1, 'nominal voltage': 1, 'cell surface receptor': 1, 'wound healing': 1, 'alkenylene': 1, 'vi': 1, 'zinc': 1, 'membere heterocycloalkyl': 1, 'serum fraction': 1, 'therapeutic benefit': 1, 'rna sequence': 1, 'bone degradation': 1, 'rhinovirus': 1, 'cationic amino acid': 1, 'sealable container': 1, 'high performance': 1, 'green tea': 1, 'viral hepatitis vaccine': 1, 'antibody-specific peptide': 1, 'digital conversion': 1, 'electrical trigger': 1, 'tissue typing': 1, 'non-touch': 1, 'non-preferred codon': 1, 'image rotation correction module': 1, 'low respiratory illness': 1, 'cmir': 1, 'non-specific alkaline phosphatase': 1, 'amll': 1, 'administer cysteamine': 1, 'cell mediate immune response': 1, 'leukemias': 1, 'cellular product': 1, 'tracking': 1, 'osmotic pressure': 1, 'swab collection equipment': 1, 'antisense oligonucleotides': 1, 'chemotherapy': 1, 'germ': 1, 'precision medicine': 1, 'distilled water': 

In [16]:
print(type(b))

<class 'dict'>


In [ ]:
data.dict_der_text[ex_id]

In [ ]:
ex_id="CA2798250A1"#"CA2798250A1"#"CN101204448A"

In [ ]:
print(list(data.df_full_text.loc[data.df_full_text['PPID'] == ex_id]['ABEN']))

In [ ]:
print(list(df_kw.loc[df_kw['PPID'] == ex_id]['KEYWORD']))

In [ ]:
spacy_NLP_model = spacy.load('en_core_web_sm', disable=['ner'])

In [ ]:
sw_list= StopWords()

In [ ]:
sw_list.sw_all

In [ ]:
df_kw, df_res= obj_gen_training_data.label_data_with_noun_phrase_pipeline()

In [ ]:
df_kw.head()

In [ ]:
len(list_kw)

In [ ]:
set1=sw_list.sw_all
set2=set(list_kw)

set3=set1.intersection(set2)

In [ ]:
set3

In [ ]:
df=pd.read_csv('../../resources/data_dwpi_basic_abstract.csv', error_bad_lines=False)

In [ ]:
df.head()

In [ ]:
data.df_full_text.head()

In [ ]:
print(data.df_full_text[data.df_full_text['PPID']=='CN101246163A']['TIEN'])

In [ ]:
df_=pd.read_csv('df_np_analysis.csv')

In [ ]:
df_[df_['#np_claim']==0]